Pagal: https://keras.io/examples/nlp/neural_machine_translation_with_transformer/

Bibliotekų įdiegimas

In [ ]:
!pip install -q --upgrade rouge-score nltk
!pip install -q --upgrade keras-hub
!pip install -q --upgrade keras

import keras_hub
import nltk

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.1/792.1 kB 34.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-nlp 0.18.1 requires keras-hub==0.18.1, but you have keras-hub 0.20.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-nlp 0.18.1 requires keras-hub==0.18.1, but you have keras-hub 0.20.0 which is incompatible.


Bibliotekų importavimas

In [ ]:
import os

import pathlib
import random
import string
import re
import numpy as np

import tensorflow.data as tf_data
import tensorflow.strings as tf_strings

import keras
from keras import layers
from keras import ops
from keras.layers import TextVectorization

Duomenų failai

In [ ]:
text_file_en = "/content/QED.en-lt.en"
text_file_lt = "/content/QED.en-lt.lt"
text_file = "/content/spa.txt"

Duomenų paruošimas. Į vertimo eilutės pradžią pridedame "[start]", o į pabaigą "[end]". Viršuje kodas skirtas lietuviškam rinkiniui, apačioje - ispaniškui.

In [ ]:
# with open(text_file_en) as f:
#     lines_en = f.read().split("\n")[:-1]
# with open(text_file_lt) as f:
#     lines_lt = f.read().split("\n")[:-1]
# text_pairs = []
# for line_en, line_lt in zip(lines_en, lines_lt):
#     line_lt = "[start] " + line_lt + " [end]"
#     text_pairs.append((line_en, line_lt))

with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    eng, tgt = line.split("\t")
    tgt = "[start] " + tgt + " [end]"
    text_pairs.append((eng, tgt))


Pažiūrime kelias poras

In [ ]:
for _ in range(5):
    print(random.choice(text_pairs))

('She suddenly became famous.', '[start] Ella de repente se hizo famosa. [end]')
("Someone must've left the water running.", '[start] Alguien tiene que haberse dejado el grifo abierto. [end]')
('He fell in love with her at first sight.', '[start] Él se enamoró de ella a primera vista. [end]')
("Tom can't read all these books in one day.", '[start] Tom no puede leer todos estos libros en un día. [end]')
('I had a lovely night.', '[start] Pasé una noche agradable. [end]')


Išskiriame į treniravimo, validacijos ir testų poras.

In [ ]:
random.seed(42)
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

118964 total pairs
83276 training pairs
17844 validation pairs
17844 test pairs


Apsirašome hyperparametrus.

In [ ]:
vocab_size = 50000
sequence_length = 30
epochs = 5
embed_dim = 512
latent_dim = 1024
num_heads = 8
batch_size = 16

Atliekama teksto vektorizacija paversti žodžius į skaičius.

In [ ]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")



def custom_standardization(input_string):
    lowercase = tf_strings.lower(input_string)
    return tf_strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")


eng_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
    standardize=custom_standardization,
)
tgt_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_eng_texts = [pair[0] for pair in train_pairs]
train_tgt_texts = [pair[1] for pair in train_pairs]
eng_vectorization.adapt(train_eng_texts)
tgt_vectorization.adapt(train_tgt_texts)

Suformatuojame duomenų rinkinius. "format_dataset" - pritaiko vektorizaciją ir grąžina "encoder_inputs" įvestį encoderiui, "decoder_inputs" - tikslo sakinys be paskutinio žodžio (kad prognozuotų sekantį), "tgt[:, 1:]" - tikslai, kurios bandys prognozuoti.

"make_dataset" - sukuria TensorFlow Dataset objektą iš sąrašų.

In [ ]:

def format_dataset(eng, tgt):
    eng = eng_vectorization(eng)
    tgt = tgt_vectorization(tgt)
    return (
        {
            "encoder_inputs": eng,
            "decoder_inputs": tgt[:, :-1],
        },
        tgt[:, 1:],
    )


def make_dataset(pairs):
    eng_texts, tgt_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    tgt_texts = list(tgt_texts)
    dataset = tf_data.Dataset.from_tensor_slices((eng_texts, tgt_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.cache().shuffle(2048).prefetch(16)


train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

Pažiūrime į gautus shape

In [ ]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (16, 30)
inputs["decoder_inputs"].shape: (16, 30)
targets.shape: (16, 30)


TransformerEncoder klasė:

embed_dim: Nurodo vektorių dydį, kuriuo žodžiai yra reprezentuojami.

dense_dim: dense sluoksnio dimensija.

num_heads: multi-head attention mechanizmo galvų skaičius.
attention = layers.MultiHeadAttention(...): Inicijuoja kelių galvų attention sluoksnį. Šis sluoksnis leidžia modeliui vienu metu atkreipti dėmesį į skirtingas įvesties sekos dalis. key_dim=embed_dim nurodo keys and queries dimensiją.

dense_proj = keras.Sequential(...): Inicijuoja dense su ReLU aktyvacija. Šis sluoksnis apdoroja attention sluoksnio išvestį.
layernorm_1 = layers.LayerNormalization() ir layernorm_2 = layers.LayerNormalization(): Inicijuoja normalizavimo sluoksnius.

supports_masking = True: Nurodo, kad šis sluoksnis palaiko maskavimą, kas yra svarbu tvarkant kintamo ilgio sekas, kad būtų ignoruojama padding.

call(self, inputs, mask=None): apibrėžia, kaip duomenys apdorojami per enkoderio sluoksnį.

if mask is not None: padding_mask = ops.cast(mask[:, None, :], dtype="int32"): pritaiko masking

attention_output = self.attention(...): Apskaičiuojama attention išvestis.        proj_input = self.layernorm_1(inputs + attention_output): Prie įvesčių pridedama attention ir atliekamas normalizavimas.

proj_output = self.dense_proj(proj_input): dense sluoksniai apdoroja normalizuotą išvestį.

return self.layernorm_2(proj_input + proj_output): Prie normalizuotos įvesties pridedama dense išvestis ir atliekamas galutinis normalizavimas.

---------------------

PositionalEmbedding klasė:

sequence_length: Maksimalus sekos ilgis.

vocab_size: Žodyno dydis.

embed_dim: embedding dimensija.

token_embeddings = layers.Embedding(...): Konvertuoja žodžių indeksus į dense vektorius.

position_embeddings = layers.Embedding(...): Inicijuoja embedding sluoksnį pozicijoms. Konvertuoja pozicijos indeksus (0, 1, 2, ...) į dense vektorius.

call(self, inputs): apibrėžia, kaip apskaičiuojami poziciniai embedding ir kaip jie sujungiami su žodžių embedding.

length = ops.shape(inputs)[-1]: Gaunamas dabartinės įvesties sekos ilgis.

positions = ops.arange(0, length, 1): Sukuriama pozicijų seka (0, 1, 2, ..., ilgis-1).

embedded_tokens = self.token_embeddings(inputs): Apskaičiuojami žodžių embedding.

embedded_positions = self.position_embeddings(positions): Apskaičiuojami poziciniai embedding.

return embedded_tokens + embedded_positions: Prie žodžių embedding pridedami poziciniai embedding. Šis sumavimas perduoda informaciją apie kiekvieno žodžio poziciją sekoje.

compute_mask(self, inputs, mask=None): Šis metodas sukuria mask, kuri nurodo, kurie elementai sekoje nėra padding. Tai svarbu, kad dėmesio mechanizmas ignoruotų padding.

-------------------


TransformerDecoder klasė:

attention_1 = layers.MultiHeadAttention(...): Pirmas attention sluoksnys (Masked Multi-Head Attention). Jis veikia tik su target kalba, bet yra maskuotas, kad būtų išvengta "žiūrėjimo į ateitį" (causal masking).

attention_2 = layers.MultiHeadAttention(...): Antras attention sluoksnys (Encoder-Decoder Attention). Jis veikia su target kalba (kaip užklausos) ir enkoderio išvestimi (kaip raktai ir vertės). Leidžia atkreipti dėmesį į šaltinio kalbos dalis.

dense_proj = keras.Sequential(...): dense sluoksniai.

layernorm_1, layernorm_2, layernorm_3: Normalizavimo sluoksniai.

supports_masking = True: Nurodo, kad sluoksnis palaiko maskavimą.

call(self, inputs, mask=None): Šis metodas apibrėžia, kaip duomenys apdorojami per dekoderio sluoksnį.

inputs, encoder_outputs = inputs: priima iki šiol sugeneruotą kalbos seką (inputs) ir enkoderio išvestį (encoder_outputs).

causal_mask = self.get_causal_attention_mask(inputs): Sukuriama causal mask, kad nežiūrėtų į ateitį.

if mask is None: ... else: inputs_padding_mask, encoder_outputs_padding_mask = mask: Tvarkomos padding tiek tikslo kalbos sekai, tiek enkoderio išvesčiai.

attention_output_1 = self.attention_1(...): Atliekamas pirmas casual attention skaičiavimas sekoje.

out_1 = self.layernorm_1(inputs + attention_output_1): Prie sekos pridedama attention išvestis ir normalizuojama.

attention_output_2 = self.attention_2(...): Atliekamas antras (enkoderis-dekoderis) attention skaičiavimas, naudojant out_1 ir encoder_outputs.

out_2 = self.layernorm_2(out_1 + attention_output_2): Prie out_1 pridedama antro attention sluoksnio išvestis ir normalizuojama.

proj_output = self.dense_proj(out_2): dense sluoksniai apdoroja normalizuotą išvestį.

return self.layernorm_3(out_2 + proj_output): Prie out_2 pridedama dense sluoksnių išvestis ir atliekamas galutinis normalizavimas.

get_causal_attention_mask(self, inputs): Šis metodas sukuria casual mask


In [ ]:
import keras.ops as ops


class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(dense_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = ops.cast(mask[:, None, :], dtype="int32")
        else:
            padding_mask = None

        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "dense_dim": self.dense_dim,
                "num_heads": self.num_heads,
            }
        )
        return config


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = ops.shape(inputs)[-1]
        positions = ops.arange(0, length, 1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return ops.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "sequence_length": self.sequence_length,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(latent_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        inputs, encoder_outputs = inputs
        causal_mask = self.get_causal_attention_mask(inputs)

        if mask is None:
            inputs_padding_mask, encoder_outputs_padding_mask = None, None
        else:
            inputs_padding_mask, encoder_outputs_padding_mask = mask

        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask,
            query_mask=inputs_padding_mask,
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            query_mask=inputs_padding_mask,
            key_mask=encoder_outputs_padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = ops.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = ops.arange(sequence_length)[:, None]
        j = ops.arange(sequence_length)
        mask = ops.cast(i >= j, dtype="int32")
        mask = ops.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = ops.concatenate(
            [ops.expand_dims(batch_size, -1), ops.convert_to_tensor([1, 1])],
            axis=0,
        )
        return ops.tile(mask, mult)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "latent_dim": self.latent_dim,
                "num_heads": self.num_heads,
            }
        )
        return config


Paruošiame galutinį modelį

In [ ]:
encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)([x, encoder_outputs])
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

transformer = keras.Model(
    {"encoder_inputs": encoder_inputs, "decoder_inputs": decoder_inputs},
    decoder_outputs,
    name="transformer",
)

Treniruojame modelį

In [ ]:
transformer.summary()
transformer.compile(
    keras.optimizers.AdamW(),
    loss=keras.losses.SparseCategoricalCrossentropy(ignore_class=0),
    metrics=["accuracy"],
)
transformer.fit(train_ds, epochs=epochs, validation_data=val_ds)

Model: "transformer"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_embeddi… │ (None, None, 512) │ 25,615,360 │ encoder_inputs[0… │
│ (PositionalEmbeddi… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, None)      │          0 │ encoder_inputs[0… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_embeddi… │ (None, None, 512) │ 25,615,360 │ decoder_inputs[0… │
│ (PositionalEmbeddi… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_encoder │ (None, None, 512) │  9,453,568 │ positional_embed… │
│ (TransformerEncode… │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_1         │ (None, None)      │          0 │ decoder_inputs[0… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_decoder │ (None, None, 512) │ 17,856,000 │ positional_embed… │
│ (TransformerDecode… │                   │            │ transformer_enco… │
│                     │                   │            │ not_equal_1[0][0… │
│                     │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, None, 512) │          0 │ transformer_deco… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, None,      │ 25,650,000 │ dropout_3[0][0]   │
│                     │ 50000)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 104,190,288 (397.45 MB)

 Trainable params: 104,190,288 (397.45 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
5205/5205 ━━━━━━━━━━━━━━━━━━━━ 741s 137ms/step - accuracy: 0.0337 - loss: 6.6040 - val_accuracy: 0.0344 - val_loss: 6.1872
Epoch 2/5
5205/5205 ━━━━━━━━━━━━━━━━━━━━ 719s 138ms/step - accuracy: 0.0353 - loss: 6.1486 - val_accuracy: 0.0354 - val_loss: 6.1530
Epoch 3/5
5205/5205 ━━━━━━━━━━━━━━━━━━━━ 722s 134ms/step - accuracy: 0.0376 - loss: 6.0550 - val_accuracy: 0.0416 - val_loss: 5.9376
Epoch 4/5
5205/5205 ━━━━━━━━━━━━━━━━━━━━ 699s 134ms/step - accuracy: 0.0400 - loss: 5.9207 - val_accuracy: 0.0419 - val_loss: 5.8613
Epoch 5/5
5205/5205 ━━━━━━━━━━━━━━━━━━━━ 716s 138ms/step - accuracy: 0.0425 - loss: 5.8242 - val_accuracy: 0.0434 - val_loss: 5.8557


Paduodame sakinį vertimui, gauname rezultatą

In [ ]:
tgt_vocab = tgt_vectorization.get_vocabulary()
tgt_index_lookup = dict(zip(range(len(tgt_vocab)), tgt_vocab))
max_decoded_sentence_length = 20


def decode_sequence(input_sentence):
    tokenized_input_sentence = eng_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = tgt_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer(
            {
                "encoder_inputs": tokenized_input_sentence,
                "decoder_inputs": tokenized_target_sentence,
            }
        )
        sampled_token_index = ops.convert_to_numpy(
            ops.argmax(predictions[0, i, :])
        ).item(0)
        sampled_token = tgt_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
    return decoded_sentence


test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(30):
    input_sentence = random.choice(test_eng_texts)
    translated = decode_sequence(input_sentence)
    translated = translated.replace("[PAD]", "").replace("[start]", "").replace("[end]", "").strip()
    print(f"{input_sentence} | {translated}")

Do you still think you'll be able to attend the meeting? | no no es
It looks as though this summer will be cold again. | ella
Wait one second. | el el
I don't know what Tom knows, but Tom knows what I know. | tom no no
He telephoned me again and again. | Él es es
A magnet can pick up and hold many nails at a time. | Él un un
You're going in the wrong direction. | Él es es
Tom and Mary are still arguing. | tom no tom
What's your favorite self-help book? | el el
I decided to go to college to major in English. | no no no
In my city, there is no school for learning Esperanto. | no no no
Don't deceive him. | 
His whole family is like that. | ¿qué ¿qué ¿qué
He used pigeons in his experiment. | Él un un
Tom should definitely ask for Mary's opinion. | tom es es
Do you like Mexican food? | es es es
I only know that if I don't take this medicine every day, I'll die. | no no no
It's in plain sight. | el el
His novels are too deep for me. | el el
This is quite tasty. | es es es
Why didn't you tell

Atliekame matavimus

In [ ]:

from nltk.translate.bleu_score import sentence_bleu

rouge_1 = keras_hub.metrics.RougeN(order=1)
rouge_2 = keras_hub.metrics.RougeN(order=2)

bleu_scores = []

for test_pair in test_pairs[:30]:
    input_sentence = test_pair[0]
    reference_sentence = test_pair[1]

    translated_sentence = decode_sequence(input_sentence)
    translated_sentence = translated_sentence.replace("[PAD]", "").replace("[start]", "").replace("[end]", "").strip() # Use lowercase "[start]" and "[end]" based on the decode_sequence function

    rouge_1.update_state(reference_sentence, translated_sentence)
    rouge_2.update_state(reference_sentence, translated_sentence)

    reference = [reference_sentence.split()]
    candidate = translated_sentence.split()
    bleu_scores.append(sentence_bleu(reference, candidate))

print("ROUGE-1 Score: ", rouge_1.result())
print("ROUGE-2 Score: ", rouge_2.result())

print("Average BLEU Score: ", sum(bleu_scores) / len(bleu_scores))

/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

ROUGE-1 Score:  {'precision': <tf.Tensor: shape=(), dtype=float32, numpy=0.1833333522081375>, 'recall': <tf.Tensor: shape=(), dtype=float32, numpy=0.04875781759619713>, 'f1_score': <tf.Tensor: shape=(), dtype=float32, numpy=0.07424840331077576>}
ROUGE-2 Score:  {'precision': <tf.Tensor: shape=(), dtype=float32, numpy=0.01666666753590107>, 'recall': <tf.Tensor: shape=(), dtype=float32, numpy=0.0025641026441007853>, 'f1_score': <tf.Tensor: shape=(), dtype=float32, numpy=0.004444444552063942>}
Average BLEU Score:  2.4087026308951578e-233
